In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
from mpl_toolkits.basemap import Basemap

import glob
import os
import csv
from pathlib import Path

import yagmail
import requests

import schedule
import time

In [ ]:
# Download a new datafile, time stamp it, and compare it to the last downloaded datafile

def pollData(): 

# download datafile as a DF

    earthquake = pd.read_csv('http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.csv')

# create a date_time stamp (global)
    
    global dt_stamp

    dt_stamp=datetime.now().strftime("%m%d%Y_%H%M%S")

# save the file as a .csv with a unique date_time stamp (in the working directory)

    earthquake.to_csv(f"eq_csv_{dt_stamp}.csv") 
    
# Get all the relevant .csv files (previously downloaded earthquake datafiles) in the downlaod directory and get their
# creation date times

    date_list=[]
    file_list=[]
    for path in Path().glob("eq*.csv"):   #Path() is empty because we're getting file from the current working directory
        create_datetime = os.stat(path).st_ctime
        date_list.append(datetime.fromtimestamp(create_datetime))
        file_list.append(path)
             
    if len(file_list) == 1: # If it's the first download
                  
        f1 = file_list[-1]
        
        newl=[]
        with open(f1,encoding = 'utf-8') as f:
            reader = csv.reader(f)
            for row in reader:      
                newl.append(row)
           
        for i in newl:
            i.pop(0)
       
        notify_data=[a for a in newl] 
        
        notify_data.pop(0) # we strip the headers here so that both notify_data versions in the if/else match up for the getData() function 
              
    else:
        
        # Assign the last two positions of 'file_list' and put them in two seperate lists
        # This corresponds to the two latest earthquake datafile downloads 
        
        f1 = file_list[-1]  # the latest download
        f2 = file_list[-2]  # the previous download
    
        newl=[]
        with open(f1,encoding = 'utf-8') as f:
            reader = csv.reader(f)
            for row in reader:      
                newl.append(row)        
        
        oldl=[]
        with open(f2, encoding = 'utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                oldl.append(row)   
        
       # Get rid of the the row numbers (the first element) in the lists so that they won't interfere when we compare 
       # the two lists against each other

        for i in newl:
            i.pop(0)
    
        for i in oldl:
            i.pop(0)        
            
        # Compare the two lists and extract only new data (earthquakes) into a new 'notify_data' list (using a list comprehension)
                        
        notify_data=[a for a in newl if a not in [b for b in oldl]] 
            
# If notify_data isn't empty then save it to disk, and call getUpdate() to create a 
# visualization, and continue the process. Otherwise, delete the redundant datafile download.

# headers, quotes around values in the 'place' feature and other things, are getting stripped as undesired side 
#   effects of the comparison, so we fix that first. Also have to add the row numbers back in, so that 
#   everything will match up when we write the .csv to disk to have the exact same formating as the original download that the differences 
#   were extracted from. 
 
    if len(notify_data) != 0:
        
        # get the headers and clean them up.       
        new_headers = str(newl[0]) 
        new_headers = new_headers.replace('[',',').replace(']','').replace("'",'').replace(' ','') 
        new_headers = new_headers.split(",")
                   
        # add quotes back to the values in the 'place' feature
        for i in notify_data: 
            i[13] = '"' + i[13] + '"'
         
        # add index numbers to data rows
        for idx, i in enumerate(notify_data):
            i.insert(0,idx)
        
        # add headers back in, and save as .csv to disk
        notify_data.insert(0, new_headers) 
                        
        with open('notify_data_' + dt_stamp + '.csv' , 'w', encoding="utf-8") as f:
            for row in notify_data:
                for x in row:
                    f.write(str(x) + ',')
                
                f.write('\n')
               
        # call the get update function     
        getUpdate(notify_data)  
         
    else:
        print("No Update")  
                
        #delete the redundant datafile download    
        files = glob.glob("eq*.csv")
        os.unlink(files[-1])       

In [ ]:
# Function that visualises update and saves file for email attachment

def getUpdate(notify_data):
   
    fig = plt.figure(figsize=(45,30))
    ax = fig.add_subplot(1,1,1)
    
# remove the column numbers again so they don't interfere with the dataframe index

    for i in notify_data:
        i.pop(0)

# Create a DF fron notify_data list
    notify_DF = pd.DataFrame(notify_data[1:], columns=notify_data[0])
    
# Get only the columns we want
    notify_DF = notify_DF[['time','latitude','longitude','depth','mag','place']]
    
# Convert the location columns to float
    notify_DF[['longitude','latitude']] = notify_DF[['longitude','latitude']].astype(float)

# Now we have differences from the newest file only
# save back to a csv from a DF and put a timestamp in the filename
# Now we can visualize any new datapoints and send an email accordingly

    filename = datetime.now().strftime('notify_%d%m%Y_%H%M%S.csv')
    notify_DF.to_csv(r'c://Users//ADMIN//filename')
   
    lats=notify_DF['latitude'].to_list()
    lons=notify_DF['longitude'].to_list()
    place=notify_DF['place'].to_list()

# Visualize the Data

    m = Basemap(projection='mill',area_thresh= 10)

    m.drawcoastlines(color='black', linewidth=0.25)
    m.fillcontinents(color='coral',lake_color='aqua')
    m.drawmapboundary(fill_color='aqua')

    x,y = m(lons,lats)
    m.scatter(x,y, marker='o', color='black', s = 100, alpha=1)

    date = datetime.utcnow()
    plt.title("Significant Earthquakes update as of UTC %s" % date.strftime("%b %d %Y %H:%M"),fontsize = 30, fontweight='bold')

    # create map legend 
    
    from matplotlib.patches import Rectangle
    hndls = [] 
    for i in range(len(notify_DF.index)):
    #hndls.append(Rectangle((0, 0), 0, 0, color = 'black', alpha=0.5)) 
        hndls.append(plt.Circle((0,0),5, color = 'black'))
  
    ax.legend(hndls, notify_DF.place, loc = 'lower right', fontsize = 30, edgecolor = 'black', facecolor = 'white', handlelength = 1, framealpha = .4 )    

    # Save visualization, with a datetime stamp             
    plt.savefig('c://Users//ADMIN//email_project//update_' + dt_stamp + '.png') 
    
    # get all the images in the image directory and assign the latest one, by timestamp, to 'latest update' variable
    
    update_list=[] 
    for path in Path().glob("email_project//*.png"):   
        update_list.append(path) 
    
    global latest_update   
    latest_update = max(update_list)  
    
    sendEmail()
    
    plt.show()     

In [ ]:
# Send Email using Yagmail

def sendEmail():
     
    file_list2=[]
    for path in Path().glob('email_project/*.png'): 
        file_list2.append(path)

    try:
    
    #initializing the server connection
    
    #two factor + app password
        yag = yagmail.SMTP('gmail address goes here', 'app password goes here')
    
    #sending the email
    
        to = 'test@domain.com'  # target email goes here (in quotes)
        subject = 'Update'
        content = 'Here is the latest update'
        attachments= max(file_list2)
                 
        yag.send(to, subject, content, attachments)
        print("Email sent successfully.")
    except:
        print("Error, email was not sent")  
        
        # call the sendsms to notify of email update
        
        sendsms()

In [ ]:
# Send SMS using Textbelt

def sendsms():

    resp = requests.post('https://textbelt.com/text', {
      'phone': '1111111111',   # target phone number goes here (in quotes)
      'message': 'Earthquakes update sent to email',
      'key': 'textbelt',
    })
    print(resp.json())

In [ ]:
# Set an interval to check for updates (call the 'pollData' function)

schedule.every(30).seconds.do(pollData)

while True:
    schedule.run_pending()